In [1]:
# necessary libraries for prediciton
import utils
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVR

ModuleNotFoundError: No module named 'sklearn'

# Prediction

* Predictions are done in this notebook.
* It is also useful to compare how serveral algorithms perform against one another.

In [2]:
dataset =  utils.read_csv_to_df('dataset/preprocessed_data.csv')

In [27]:
# Useful MACROS
STATUS_COL = 7
K_FOLD_NUM_SPLITS = 5
SEED = 42

In [28]:
# Setting X and Y
X = dataset.iloc[:, 0:STATUS_COL-1]
Y = dataset.iloc[:, STATUS_COL]

IndexError: single positional indexer is out-of-bounds

In [26]:
# Applying Cross validation
scores = []
best_svr = SVR(kernel='rbf')
cv = KFold(n_splits=K_FOLD_NUM_SPLITS, random_state=SEED, shuffle=False)
for train_index, test_index in cv.split(X):
    print("Train Index: ", train_index, "\n")
    print("Test Index: ", test_index)

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    best_svr.fit(X_train, y_train)
    scores.append(best_svr.score(X_test, y_test))


NameError: name 'SVR' is not defined

FileNotFoundError: [Errno 2] File b'housing.csv' does not exist: b'housing.csv'